In [1]:
import numpy as np
from numpy import linalg as LA
from sklearn import metrics
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from time import process_time
from scipy.stats import norm
from scipy.stats import multivariate_normal as mvn


## Preprocessing, Initialisation 

In [2]:
x, y = load_digits(return_X_y=True)

# Normaliseing the data for numerical reasons
x = x/16.0

# Splitting
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

## EM-Algorithm

In [3]:
class GNBC:
    def __init__(self, nb_classes, example_size, itr=10):
        self.nb_classes = nb_classes
        self.size = example_size
        self.priors = {k:1/self.nb_classes for k in range(self.nb_classes)}
        self.mean = np.zeros((nb_classes,self.size))
        self.var = np.zeros((nb_classes,self.size))
        self.eps = 0.01
        self.itr = itr
        self.classes = np.zeros(self.nb_classes)
        self.subset = {}
        
    
 
    
    def fit(self,X,y):

        rand_init = np.random.rand(10,64)
        self.mean = {k: np.random.uniform(1,size=X[0].size) for k in range(self.nb_classes)}
        self.var = {k:np.random.uniform(0.5,size=X[0].size) + self.eps for k in range(self.nb_classes)}
        r = np.zeros((len(X),self.nb_classes))
        t = 1
        while t < self.itr + 1:
            
            start = process_time() 
            # E-step
            for idx,x in enumerate(X):
                num = np.prod([norm.pdf(x, self.mean[k], np.sqrt(self.var[k]))
                      for k in range(self.nb_classes)], axis = 1)
                num = [self.priors[k]*num[k] for k in range(self.nb_classes)]
                den = np.sum(num)
                r[idx,:] = num/den    
            
            # M-step
            N, c = r.shape  
            self.priors = {c:np.sum(r[:,c])/N for c in range(self.nb_classes)}
            mean = np.zeros(len(X[0]))
            for c in range(self.nb_classes):
                mean = np.sum([r[idx,c]*x for idx, x in enumerate(X)], axis=0)/np.sum(r[:,c])
                self.mean[c] = mean
                self.var[c] = np.sum([r[idx,c]*np.diag(np.outer(x, x)) for idx, x in enumerate(X)], axis=0)/np.sum(r[:,c]) - np.diag(np.outer(mean, mean)) + self.eps
                
            end = process_time()
            print('Iteration:', t, end-start ,' seconds')
            t += 1  
    
    def repair(self,X,y):
        y_pred = self.prediction(X,False)
        for idx,pred in enumerate(y_pred):
            if pred in self.subset:
                self.subset[pred].append(y[idx])
            else:
                self.subset[pred] = [y[idx]]
#         cluster_label = {cluster:label}        
        self.cluster_label = {i:np.argmax(np.bincount(self.subset[i])) for i in range(self.nb_classes)}
   
    def prediction(self,X,repaired=True):
        pred = np.zeros((len(X),self.nb_classes))
        for k in range(self.nb_classes):
            pred[:,k] = mvn.logpdf(X, mean=self.mean[k], cov=self.var[k], allow_singular=True) + np.log(self.priors[k])      
        
        if repaired:
              return [self.cluster_label[p] for p in np.argmax(pred, axis=1)]
        else:
            return np.argmax(pred, axis=1)
    
 

In [4]:
clf = GNBC(10,64,100)
clf.fit(X_train,y_train)

Iteration: 1 2.171875  seconds
Iteration: 2 2.203125  seconds
Iteration: 3 1.90625  seconds
Iteration: 4 1.890625  seconds
Iteration: 5 1.890625  seconds
Iteration: 6 1.9375  seconds
Iteration: 7 1.875  seconds
Iteration: 8 1.828125  seconds
Iteration: 9 2.015625  seconds
Iteration: 10 1.9375  seconds
Iteration: 11 1.890625  seconds
Iteration: 12 1.875  seconds
Iteration: 13 1.984375  seconds
Iteration: 14 2.046875  seconds
Iteration: 15 2.046875  seconds
Iteration: 16 2.171875  seconds
Iteration: 17 2.015625  seconds
Iteration: 18 1.90625  seconds
Iteration: 19 1.875  seconds
Iteration: 20 1.828125  seconds
Iteration: 21 1.859375  seconds
Iteration: 22 1.890625  seconds
Iteration: 23 1.84375  seconds
Iteration: 24 1.921875  seconds
Iteration: 25 1.875  seconds
Iteration: 26 1.828125  seconds
Iteration: 27 1.875  seconds
Iteration: 28 1.875  seconds
Iteration: 29 1.796875  seconds
Iteration: 30 1.859375  seconds
Iteration: 31 1.8125  seconds
Iteration: 32 1.84375  seconds
Iteration: 33

In [5]:
y_pred = clf.prediction(X_train,repaired=False)
lbs = [0,1,2,3,4,5,6,7,8,9]
print("Classification report GNBC digits (unrepaired):\n%s\n"
          % (metrics.classification_report(y_train, y_pred,labels=lbs)))
print("Confusion matrix GNBC digits:\n%s" % metrics.confusion_matrix(y_train, y_pred,labels=lbs))

Classification report GNBC digits (unrepaired):
              precision    recall  f1-score   support

           0       1.00      0.98      0.99       125
           1       0.48      0.33      0.39       132
           2       0.00      0.00      0.00       130
           3       0.01      0.01      0.01       129
           4       0.00      0.00      0.00       121
           5       0.00      0.00      0.00       116
           6       0.00      0.00      0.00       128
           7       0.00      0.00      0.00       124
           8       0.00      0.00      0.00       131
           9       0.01      0.02      0.01       121

    accuracy                           0.14      1257
   macro avg       0.15      0.13      0.14      1257
weighted avg       0.15      0.14      0.14      1257


Confusion matrix GNBC digits:
[[123   0   0   0   0   0   0   2   0   0]
 [  0  44   0   1  14   3   0   0   0  70]
 [  0   7   0   0 120   0   0   0   0   3]
 [  0   1 103   1   2   0  12   0

In [6]:
clf.repair(X_train,y_train)
for k in range(10):
    print('Cluster: '+ str(k) + ' => label: ' + str(clf.cluster_label[k]))


Cluster: 0 => label: 0
Cluster: 1 => label: 1
Cluster: 2 => label: 3
Cluster: 3 => label: 5
Cluster: 4 => label: 2
Cluster: 5 => label: 6
Cluster: 6 => label: 9
Cluster: 7 => label: 4
Cluster: 8 => label: 7
Cluster: 9 => label: 8


In [7]:
y_pred = clf.prediction(X_train,repaired=True)
print("Classification report GNBC digits (repaired):\n%s\n"
          % (metrics.classification_report(y_train, y_pred)))
print("Confusion matrix GNBC digits:\n%s" % metrics.confusion_matrix(y_train, y_pred))

Classification report GNBC digits (repaired):
              precision    recall  f1-score   support

           0       1.00      0.98      0.99       125
           1       0.48      0.33      0.39       132
           2       0.86      0.92      0.89       130
           3       0.97      0.80      0.88       129
           4       0.97      0.91      0.94       121
           5       0.94      0.70      0.80       116
           6       0.97      0.98      0.98       128
           7       0.87      0.94      0.90       124
           8       0.46      0.62      0.53       131
           9       0.58      0.76      0.66       121

    accuracy                           0.79      1257
   macro avg       0.81      0.79      0.80      1257
weighted avg       0.81      0.79      0.79      1257


Confusion matrix GNBC digits:
[[123   0   0   0   2   0   0   0   0   0]
 [  0  44  14   0   0   1   3   0  70   0]
 [  0   7 120   0   0   0   0   0   3   0]
 [  0   1   2 103   0   1   0   3  

In [8]:
y_pred = clf.prediction(X_test,True)
print("Classification report GNBC digits (repaired):\n%s\n"
          % (metrics.classification_report(y_test, y_pred)))
print("Confusion matrix GNBC digits:\n%s" % metrics.confusion_matrix(y_test, y_pred))

Classification report GNBC digits (repaired):
              precision    recall  f1-score   support

           0       1.00      0.94      0.97        53
           1       0.54      0.30      0.38        50
           2       0.77      0.94      0.85        47
           3       0.97      0.72      0.83        54
           4       0.96      0.92      0.94        60
           5       0.97      0.59      0.74        66
           6       0.98      0.96      0.97        53
           7       0.83      0.95      0.88        55
           8       0.45      0.65      0.53        43
           9       0.52      0.80      0.63        59

    accuracy                           0.78       540
   macro avg       0.80      0.78      0.77       540
weighted avg       0.81      0.78      0.78       540


Confusion matrix GNBC digits:
[[50  0  0  0  2  0  0  0  0  1]
 [ 0 15 10  0  0  0  0  0 25  0]
 [ 0  2 44  0  0  0  0  0  1  0]
 [ 0  1  2 39  0  0  0  2  3  7]
 [ 0  1  0  0 55  0  0  4  0  0]

In [9]:
from sklearn.cluster import KMeans
clust = KMeans(10).fit(X_test)
y_test = clust.labels_
print("Classification report K-means digits:\n%s\n"
          % (metrics.classification_report(y_test, y_pred)))
print("Confusion matrix K-means digits:\n%s" % metrics.confusion_matrix(y_test, y_pred))

Classification report K-means digits:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        55
           1       0.00      0.00      0.00        51
           2       0.11      0.09      0.09        70
           3       0.00      0.00      0.00        47
           4       0.00      0.00      0.00        51
           5       0.00      0.00      0.00        55
           6       0.00      0.00      0.00        75
           7       0.83      0.88      0.85        59
           8       0.00      0.00      0.00        23
           9       0.02      0.04      0.03        54

    accuracy                           0.11       540
   macro avg       0.10      0.10      0.10       540
weighted avg       0.11      0.11      0.11       540


Confusion matrix K-means digits:
[[ 0  0  0  0 52  0  0  3  0  0]
 [ 0  0 51  0  0  0  0  0  0  0]
 [ 0  3  6 35  0  0  0  2 10 14]
 [ 0  1  0  0  1  0  0  0 45  0]
 [ 0  1  0  0  0 40  0  6  2  2]
 [ 0